# Análisis de cliente terpel

## Importaciones

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

## Cargue de bases

In [2]:
from pathlib import Path
import re
import os

# Construir ./data/costeos_detalle_terpel.xlsx concatenando todos los .xlsx en ./downloads/costeos_detalle

def consolidar_cost_det():
    src_dir = Path('./downloads/costeo_detallado')
    out_path = Path('./data/costeos_detalle_terpel.xlsx')
    out_path.parent.mkdir(parents=True, exist_ok=True)

    files = sorted(src_dir.glob('*.xls'))
    if not files:
        raise FileNotFoundError(f"No se encontraron archivos .xls en {src_dir.resolve()}")

    frames = []
    for f in files:
        try:
            part = pd.read_excel(f)
        except Exception as e:
            print(f"Error leyendo {f.name}: {e}. Se omite este archivo.")
            continue

        # Extraer un id numérico del nombre del archivo (primer grupo de dígitos encontrado)
        # Tomar el nombre del archivo sin extensión como id (si es numérico).
        # Si no es un entero exacto, buscar el primer grupo de dígitos en el nombre.
        stem = f.stem.strip()
        m = re.match(r'^(\d+)$', stem)
        if not m:
            m = re.search(r'(\d{3,})', stem)
        id_val = int(m.group(1)) if m else None

        # Añadir/sobrescribir columna 'Id.' con el id extraído
        part['Id.'] = id_val
        frames.append(part)

    if not frames:
        raise RuntimeError("No se pudo leer ningún archivo válido para concatenar.")

    df_all = pd.concat(frames, ignore_index=True)

    # Guardar el archivo final para que las celdas posteriores lo puedan leer
    df_all.to_excel(out_path, index=False)
    print(f"Concatenados {len(frames)} archivos. Resultado: {df_all.shape[0]} filas x {df_all.shape[1]} columnas -> {out_path}")

In [3]:
from pathlib import Path

_out_path = Path('./data/costeos_detalle_terpel.xlsx')
if not _out_path.exists():
    try:
        consolidar_cost_det()
    except Exception as e:
        raise RuntimeError(f"No se pudo consolidar costeos detalle: {e}") from e

# (re)leer para asegurarnos de tener el archivo actualizado
df_costeos_detalle = pd.read_excel(_out_path)   

In [4]:
df_ops_det = pd.read_excel('./data/ops_detalle_terpel.xlsx')
df_costeos = pd.read_excel('./data/costeos_terpel.xlsx')

# Asignación de tipos de datos
for df in (df_ops_det, df_costeos, df_costeos_detalle):
    dcols = [c for c in (globals().get('date_cols') or list(df.columns)) if ('fecha' in c.lower() or 'date' in c.lower() or 'creado' in c.lower()) and c in df.columns]
    ncols = [c for c in (globals().get('num_cols') or df.select_dtypes(include=['number']).columns.tolist()) if c in df.columns]
    scols = [c for c in (globals().get('str_cols') or [c for c in df.columns if c not in dcols + ncols]) if c in df.columns]
    if dcols: df[dcols] = df[dcols].apply(pd.to_datetime, errors='coerce')
    if ncols: df[ncols] = df[ncols].apply(pd.to_numeric, errors='coerce')
    if scols: df[scols] = df[scols].astype('string')

c:\Users\Analitycs Data\OneDrive - MOLT\Documentos\MOLT\Analytics\Forecast Producción\Terpel\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\Analitycs Data\OneDrive - MOLT\Documentos\MOLT\Analytics\Forecast Producción\Terpel\.venv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\Analitycs Data\AppData\Local\Temp\ipykernel_14180\308708442.py:9: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  if dcols: df[dcols] = df[dcols].apply(pd.to_datetime, errors='coerce')


In [5]:
df_ops_det.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1091 entries, 0 to 1090
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Num-OP                   1091 non-null   int64         
 1   Secuencia                1091 non-null   int64         
 2   OP Det                   1091 non-null   string        
 3   ClienteNombre            1091 non-null   string        
 4   Productos                1091 non-null   string        
 5   Categoría Proc           936 non-null    string        
 6   Detalle                  1030 non-null   string        
 7   Colores                  945 non-null    string        
 8   Total Pieza              1070 non-null   float64       
 9   Producir                 1091 non-null   int64         
 10  Cantidad Remitida        1091 non-null   int64         
 11  Precio Unitario Sin IVA  1091 non-null   int64         
 12  Total Precio Sin IVA     1091 non-

## Limpieza de bases

In [6]:
# Op detalle
# Se eliminan muestras y OPs sin costeo asignado
df_ops_det = df_ops_det[(df_ops_det['Muestra'] == 'False') & (df_ops_det['Costeo_Producto'].notna())]
df_ops_det.drop(columns=['Estado OP', 'Ficha Técnica Producto', 'Id.', 'Id Costeo Producto', 'Muestra', 'OS', 'Fecha Promesa'], inplace=True, errors='ignore')

# Costeos
df_costeos.drop(columns=['Secuencia', 'Ficha Técnica Producto', 'Ficha Técnica Producto', 'Colores', 'Es Duplicado', 'Fijar Precio Venta', 'Costo Negociado Sin IVA', 'Negociado con', 'Categoría'], inplace=True, errors='ignore')

# Costeos detalle
df_costeos_detalle.drop(columns=['Costeo Detalle', 'Secuencia', 'Detalle', 'Costo SIN IVA', 'Generado Auto'], inplace=True, errors='ignore')

In [7]:
# normalización a lowercase de todos los valores de columnas de texto
for df in (df_ops_det, df_costeos, df_costeos_detalle):
    for c in df.select_dtypes(include=['string', 'object']).columns:
        df[c] = df[c].str.lower().str.strip()

### OPs Detalle

In [8]:
# Limpieza categorias
# Eliminacion de caracteres especiales y espacios en blanco
df_ops_det['Categoría Proc'] = df_ops_det['Categoría Proc'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True).str.strip()

# reemplazos manuales
# si contiene chaqueta
df_ops_det.loc[df_ops_det['Categoría Proc'].str.contains('chaqueta', na=False), 'Categoría Proc'] = 'chaquetas'
# si contiene pantal
df_ops_det.loc[df_ops_det['Categoría Proc'].str.contains('pantal', na=False), 'Categoría Proc'] = 'pantalones'
# si contiene saco
df_ops_det.loc[df_ops_det['Categoría Proc'].str.contains('saco', na=False), 'Categoría Proc'] = 'sacos'
# si contiene sueter
df_ops_det.loc[df_ops_det['Categoría Proc'].str.contains('sueter', na=False), 'Categoría Proc'] = 'sacos'

In [9]:
df_ops_det['descripcion_concat'] = df_ops_det['Productos'].fillna('') + ' + ' + df_ops_det['Detalle'].fillna('')
df_ops_det_catna = df_ops_det[df_ops_det['Categoría Proc'].isna()]

In [10]:
# Clasificación automática de categorías faltantes
# basado en palabras clave y Gemini
categories = ['accesorios','bata','buzo','camisa','camiseta','chaleco','chaquetas','delantal','gorra','hoodie','merchandising','overol','pantalones','ropa interior','sacos','servicios','tapabocas','transporte']
_kw = {
    'accesorios':['correa','reata','termo','cartuchera','tote','bolsa'],
    'bata':['bata','laboratorio'],
    'buzo':['buzo','sudadera'],
    'camisa':['camisa','oxford'],
    'camiseta':['camiseta','polo','t-shirt','pique'],
    'chaleco':['chaleco'],
    'chaquetas':['chaqueta','rompevientos','enguatad','michellin','michelin'],
    'delantal':['delantal'],
    'gorra':['gorra'],
    'hoodie':['hoodie','hoddie'],
    'merchandising':['merchandising'],
    'overol':['overol'],
    'pantalones':['pantal','jean','dril'],
    'ropa interior':['ropa interior'],
    'sacos':['saco','sacos'],
    'servicios':['servicios','transporte flete'],
    'tapabocas':['tapabocas'],
    'transporte':['transporte']
}
def kw_classify(text):
        t = (text or '').lower()
        for cat, keys in _kw.items():
                if any(k in t for k in keys):
                        return cat
        return None

# intento con Gemini si está disponible
try:
        import os
        from google import generativeai as genai
        def gemini_classify(text):
                prompt = f"Asignar UNA sola categoría exacta de la lista: {', '.join(categories)}. Responde únicamente la categoría (una palabra), sin explicaciones.\n\nTexto: \"{text}\"\n\nCategoría:"
                r = genai.generate_text(model='gemini-2.5-flash', prompt=prompt)
                return r.candidates[0].content.strip().lower().splitlines()[0]
except Exception:
        gemini_classify = lambda t: None

mask = df_ops_det['Categoría Proc'].isna()
df_ops_det.loc[mask, 'Categoría Proc'] = (
        df_ops_det.loc[mask, 'descripcion_concat'].fillna('').astype('string')
        .apply(lambda s: kw_classify(s) or gemini_classify(s) or 'otro')
)

In [11]:
df_ops_det.groupby('Categoría Proc', dropna=False).count()['Secuencia'].sort_values(ascending=False)

Categoría Proc
camiseta         215
chaquetas        124
gorra            107
pantalones       104
delantal          93
camisa            84
accesorios        60
hoodie            33
sacos             19
merchandising     13
chaleco           13
buzo               7
tapabocas          5
ropa interior      4
otro               3
servicios          3
overol             3
bata               2
transporte         1
Name: Secuencia, dtype: int64

### Costeos detalle

In [12]:
# Extraer categoria de insumo de substring dentro de parentesis en columna 'Insumo'
df_costeos_detalle["categoria_insumo"] = (
    df_costeos_detalle["Insumo"].str.extract(r"\((.*?)\)", expand=False).str.strip()
)

# eliminar texto dentro de parentesis en columna 'Insumo'
df_costeos_detalle["Insumo"] = (
    df_costeos_detalle["Insumo"].str.replace(r"\(.*?\)", "", regex=True).str.strip()
)

# reemplazar categoria_insumo por mat.prima o insumo
df_costeos_detalle["categoria_insumo"].loc[
    (df_costeos_detalle["categoria_insumo"] == "producto/insumo/mat.prima")
    & (df_costeos_detalle["Insumo"].str.startswith("t"))
] = "mat.prima"

df_costeos_detalle["categoria_insumo"].loc[
    (df_costeos_detalle["categoria_insumo"] == "producto/insumo/mat.prima")
    & (~df_costeos_detalle["Insumo"].str.startswith("t"))
] = "insumo"

C:\Users\Analitycs Data\AppData\Local\Temp\ipykernel_14180\3837346730.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_costeos_detalle["categoria_insumo"].loc[
C:\Users\Analitycs Data\AppData\Local\Temp\ipykernel_14180\3837346730.py:1

## Unificación de bases

In [13]:
df_maestro = df_ops_det.merge(df_costeos, how='left', on='Costeo_Producto')
df_maestro = df_maestro.merge(df_costeos_detalle, how='left', on='Id.')
df_maestro = df_maestro[df_maestro['Id.'].notna()]

In [14]:
# Columnas agregadas
df_maestro['CantInsumo'] = df_maestro['Producir'] * df_maestro['Consumo']
df_maestro['CostoTotalInsumo'] = df_maestro['CantInsumo'] * df_maestro['Total']

In [15]:
# Eliminar columnas innecesarias
df_maestro.drop(columns=['Costeo_Producto', 'Productos', 'Detalle_x', 'Detalle_y', 'descripcion_concat', 'Id Costeo Producto', 'Cliente'], inplace=True, errors='ignore')

In [16]:
df_maestro.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5985 entries, 0 to 5988
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Num-OP                   5985 non-null   int64         
 1   Secuencia                5985 non-null   int64         
 2   OP Det                   5985 non-null   string        
 3   ClienteNombre            5985 non-null   string        
 4   Categoría Proc           5985 non-null   string        
 5   Colores                  5530 non-null   string        
 6   Total Pieza              5838 non-null   float64       
 7   Producir                 5985 non-null   int64         
 8   Cantidad Remitida        5985 non-null   int64         
 9   Precio Unitario Sin IVA  5985 non-null   int64         
 10  Total Precio Sin IVA     5985 non-null   int64         
 11  Creado El                5985 non-null   datetime64[ns]
 12  Producto                 5975 non-null 

In [17]:
df_costeos_detalle[df_costeos_detalle['Id.'] == 69007590]['Total'].sum()

np.int64(19460)

In [18]:
(57197-(40350+2018))/(40350+2018)

0.35000472054380666

In [19]:
df_maestro[df_maestro.columns[20:]].sample(5)

,Insumo,Color,Consumo,Unidad Medida,Total,categoria_insumo,CantInsumo,CostoTotalInsumo
4011,corte y confeccion,<NA>,1.0,<NA>,7500.0,proceso producción,124.0,930000.0
2450,crn0003-cremallera nylon eka 80cm,<NA>,1.0,unidad,2100.0,insumo,10.0,21000.0
2366,tvi0069-vioto,<NA>,1.1,metro,8250.0,mat.prima,22.0,181500.0
4734,corte y confeccion,<NA>,1.0,<NA>,7500.0,proceso producción,200.0,1500000.0
819,man0054 marquilla nylon instrucciones de lavado,<NA>,1.0,metro,60.0,insumo,4.0,240.0


## Análisis 2024

In [20]:
df_maestro_2024 = df_maestro[df_maestro['Creado El'].dt.year == 2024]

In [21]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [22]:
df_maestro_2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2270 entries, 906 to 3175
Data columns (total 28 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Num-OP                   2270 non-null   int64         
 1   Secuencia                2270 non-null   int64         
 2   OP Det                   2270 non-null   string        
 3   ClienteNombre            2270 non-null   string        
 4   Categoría Proc           2270 non-null   string        
 5   Colores                  2270 non-null   string        
 6   Total Pieza              2220 non-null   float64       
 7   Producir                 2270 non-null   int64         
 8   Cantidad Remitida        2270 non-null   int64         
 9   Precio Unitario Sin IVA  2270 non-null   int64         
 10  Total Precio Sin IVA     2270 non-null   int64         
 11  Creado El                2270 non-null   datetime64[ns]
 12  Producto                 2270 non-nul

### Volumenes de producción y participacion

Insight: estacionalidad, picos de producción, comparación entre categorías.

In [23]:
def grafica_volumenes():
    # Agrupamos por mes y categoría
    df_plot = (
        df_maestro_2024
        .assign(Mes=df_maestro_2024["Creado El"].dt.month_name())
        .groupby(["Mes", "Categoría Proc"], as_index=False)["Producir"].sum()
    )
    
    # Para que los meses salgan en orden cronológico
    orden_meses = [
        "January","February","March","April","May","June",
        "July","August","September","October","November","December"
    ]
    df_plot["Mes"] = pd.Categorical(df_plot["Mes"], categories=orden_meses, ordered=True)
    df_plot = df_plot.sort_values("Mes")

    # Gráfica interactiva
    fig = px.line(
        df_plot,
        x="Mes",
        y="Producir",
        color="Categoría Proc",
        markers=True,
        title="Volúmenes de Producción 2024 por Categoría de Producto"
    )
    
    # Personalización moderna
    fig.update_layout(
        xaxis_title="Mes",
        yaxis_title="Cantidad producida",
        legend_title="Categoría de Producto",
        hovermode="x unified",
        template="plotly_white"
    )
    
    return fig

grafica_volumenes().show()

In [41]:
def grafica_voluenes_bajo_total():
    # Agrupamos por mes y categoría
    df_plot = (
        df_maestro_2024
        .assign(Mes=df_maestro_2024["Creado El"].dt.strftime("%B"))
        .groupby(["Mes", "Categoría Proc"], as_index=False)["Producir"]
        .sum()
    )

    # Ordenar meses cronológicamente (enero a diciembre)
    meses_orden = [
        "January", "February", "March", "April", "May", "June",
        "July", "August", "September", "October", "November", "December"
    ]
    df_plot["Mes"] = pd.Categorical(df_plot["Mes"], categories=meses_orden, ordered=True)
    df_plot = df_plot.sort_values(["Mes", "Categoría Proc"])

    # Pivot para áreas apiladas
    df_pivot = df_plot.pivot(index="Mes", columns="Categoría Proc", values="Producir").fillna(0)
    df_pivot["Total"] = df_pivot.sum(axis=1)

    # Crear figura
    fig = go.Figure()

    # Áreas apiladas (categorías)
    for cat in df_pivot.columns.drop("Total"):
        fig.add_trace(go.Scatter(
            x=df_pivot.index,
            y=df_pivot[cat],
            mode="lines",
            stackgroup="one",
            name=cat,
            hoverinfo="x+y+name"
        ))

    # Línea del total
    fig.add_trace(go.Scatter(
        x=df_pivot.index,
        y=df_pivot["Total"],
        mode="lines+markers",
        line=dict(color="black", width=3),
        name="Total"
    ))

    # Ajustes visuales
    fig.update_layout(
        title="Producción Mensual 2024 Acumulada por Categoría de Producto",
        xaxis_title="Mes",
        yaxis_title="Cantidad Producida",
        hovermode="x unified",
        xaxis=dict(tickangle=45)
    )

    return fig

grafica_voluenes_bajo_total().show()

In [25]:
# ...existing code...
def resumen_unico_compacto(df=df_maestro_2024):
    meses = ["January","February","March","April","May","June","July","August","September","October","November","December"]
    g = (df.assign(Mes=df["Creado El"].dt.strftime("%B"))
         .groupby(["Mes","Categoría Proc"], as_index=False)["Producir"].sum())
    g["Mes"] = pd.Categorical(g["Mes"], categories=meses, ordered=True)
    pivot = g.pivot(index="Categoría Proc", columns="Mes", values="Producir").reindex(columns=meses, fill_value=0)

    monthly_totals_abs = pivot.sum(axis=0)
    monthly_totals = monthly_totals_abs.replace(0, np.nan)
    pct_month = pivot.div(monthly_totals, axis=1).multiply(100).round(2).fillna(0)

    totals_annual = pivot.sum(axis=1)
    total_all = totals_annual.sum()
    pct_annual = (totals_annual / total_all * 100).round(2) if total_all else pd.Series(0.0, index=totals_annual.index)

    out = pct_month.copy()
    out["Total_Anual"] = totals_annual
    out["Pct_Anual"] = pct_annual
    out = out.sort_values("Pct_Anual", ascending=False)

    # fila final con totales absolutos por mes + total anual + pct 100%
    total_row = pd.Series(index=out.columns, dtype=object)
    for m in meses:
        total_row[m] = int(monthly_totals_abs.get(m, 0))
    total_row["Total_Anual"] = int(total_all)
    total_row["Pct_Anual"] = 100.0

    out = pd.concat([out, total_row.to_frame().T])
    out.index = out.index.map(str)
    out.rename(index={out.index[-1]: "TOTAL UNIDADES"}, inplace=True)
    return out

# generar y mostrar
df_resumen_unico = resumen_unico_compacto()
display(df_resumen_unico)
# ...existing code...

Mes,January,February,March,April,May,June,July,August,September,October,November,December,Total_Anual,Pct_Anual
pantalones,48.2,56.69,0.0,49.27,10.04,11.36,7.97,10.62,40.02,15.82,0.0,8.31,224517.0,34.16
camiseta,42.54,42.97,100.0,39.72,41.14,16.16,5.98,31.03,14.19,9.55,0.0,20.51,202187.0,30.77
gorra,1.94,0.0,0.0,1.1,16.48,11.26,55.56,24.74,12.49,36.56,0.0,3.05,69079.0,10.51
accesorios,1.38,0.0,0.0,0.66,1.35,33.08,0.0,3.42,3.73,24.96,0.0,7.12,61546.0,9.37
delantal,4.87,0.0,0.0,9.01,6.78,12.45,30.08,16.05,3.48,9.76,0.0,0.0,46339.0,7.05
chaquetas,0.05,0.0,0.0,0.24,5.39,13.91,0.43,7.13,25.62,0.5,0.0,61.02,44604.0,6.79
camisa,0.89,0.0,0.0,0.0,0.85,1.77,0.0,4.89,0.47,2.05,0.0,0.0,6553.0,1.0
sacos,0.0,0.0,0.0,0.0,17.97,0.0,0.0,0.54,0.0,0.26,0.0,0.0,1140.0,0.17
buzo,0.0,0.34,0.0,0.0,0.0,0.0,0.0,1.54,0.0,0.0,0.0,0.0,542.0,0.08
bata,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.27,0.0,0.0,224.0,0.03


In [26]:
# ...existing code...
def plot_resumen_plotly(df=df_maestro_2024):
    meses = ["January","February","March","April","May","June","July","August","September","October","November","December"]
    g = (df.assign(Mes=df["Creado El"].dt.strftime("%B"))
         .groupby(["Mes","Categoría Proc"], as_index=False)["Producir"].sum())
    g["Mes"] = pd.Categorical(g["Mes"], categories=meses, ordered=True)
    pivot = g.pivot(index="Categoría Proc", columns="Mes", values="Producir").reindex(columns=meses, fill_value=0)

    monthly_totals_abs = pivot.sum(axis=0)
    month_pct = pivot.div(monthly_totals_abs.replace(0, np.nan), axis=1).multiply(100).round(2).fillna(0)

    totals_annual = pivot.sum(axis=1)
    total_all = totals_annual.sum()
    pct_annual = (totals_annual / total_all * 100).round(2) if total_all else pd.Series(0.0, index=totals_annual.index)

    # ordenar categorías por participación anual descendente
    order = pct_annual.sort_values(ascending=False).index
    month_pct = month_pct.reindex(order)
    pct_annual = pct_annual.reindex(order)

    # figura: heatmap (meses %) + barra horizontal (pct anual)
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    fig = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3],
                        specs=[[{"type":"heatmap"},{"type":"bar"}]],
                        subplot_titles=("Participación mensual (%)","Participación anual (%)"))

    fig.add_trace(go.Heatmap(
        z=month_pct.values,
        x=month_pct.columns,
        y=month_pct.index,
        colorscale="Blues",
        colorbar=dict(title="%"),
        text=(month_pct.astype(str) + "%"),
        hovertemplate="%{y}<br>%{x}: %{z}%<extra></extra>"
    ), row=1, col=1)

    fig.add_trace(go.Bar(
        x=pct_annual.values,
        y=pct_annual.index,
        orientation='h',
        text=(pct_annual.astype(str) + "%"),
        textposition='outside',
        marker_color='darkblue',
        showlegend=False
    ), row=1, col=2)

    fig.update_layout(height=800, width=1200, margin=dict(t=60))
    fig.update_yaxes(autorange="reversed", row=1, col=1)  # mantener orden visual
    fig.update_yaxes(autorange="reversed", row=1, col=2)
    return fig

plot_resumen_plotly().show()

### Materias Primas, Insumos y Servicios

Insigths:
- Heatmap qué productos consumen más cada tipo de insumo (Distribucion).
- Treemap: qué materias primas pesan más dentro de cada prenda.
- Bar chart: Insumos que concentran mayor peso en costos.

In [35]:
# === 1. Treemap / Sunburst ===
def grafica_treemap_insumos():
    fig = px.treemap(
        df_maestro_2024,
        path=["Categoría Proc", "categoria_insumo", "Insumo"],
        values="CostoTotalInsumo",
        color="categoria_insumo",
        title="Distribución de costos por producto → categoría de insumo → insumo"
    )

    # Mostrar porcentaje respecto al padre y respecto al total en el hover
    fig.update_traces(
        root_color="lightgrey",
        hovertemplate=(
            "<b>%{label}</b><br>"
            "Costo: %{value:,.0f}<br>"
            "% del padre: %{percentParent:.2%}<br>"
            "% del total: %{percentRoot:.2%}<extra></extra>"
        )
    )

    fig.update_layout(margin=dict(t=40, l=0, r=0, b=0))
    return fig


# === 2. Bar Chart (Top 10 Insumos más costosos) ===
def grafica_top_insumos():
    # Top 10 insumos por costo
    df_top_ins = (
        df_maestro_2024.groupby("Insumo", as_index=False)["CostoTotalInsumo"]
        .sum()
        .sort_values("CostoTotalInsumo", ascending=False)
        .head(20)
    )

    # Totales agregados por categoría (mat.prima e insumo)
    df_cat = (
        df_maestro_2024.groupby("categoria_insumo", as_index=False)["CostoTotalInsumo"]
        .sum()
    )
    df_cat = df_cat[df_cat["categoria_insumo"].isin(["mat.prima", "insumo"])].copy()
    # Renombrar para concatenar y marcar que son agregados de categoría
    df_cat = df_cat.rename(columns={"categoria_insumo": "Insumo"})
    df_cat["Insumo"] = df_cat["Insumo"].apply(lambda x: f"CAT: {x}")

    # Concatenar top insumos + categorías y ordenar
    df_plot = pd.concat([df_top_ins, df_cat], ignore_index=True)
    df_plot = df_plot.sort_values("CostoTotalInsumo", ascending=False).reset_index(drop=True)

    # Columna en miles de millones para el eje x y texto formateado
    df_plot["Costo_mil_millones"] = df_plot["CostoTotalInsumo"] / 1e9
    df_plot["texto"] = df_plot["CostoTotalInsumo"].apply(lambda x: f"${x:,.0f}")

    fig = px.bar(
        df_plot,
        x="Costo_mil_millones",
        y="Insumo",
        orientation="h",
        text="texto",
        title="Top 20 insumos por costo total + totales agregados (categorías)"
    )

    fig.update_traces(textposition="outside", hovertemplate="%{y}<br>Costo: %{text}<br>%{x:.2f} mil millones<extra></extra>")

    fig.update_layout(
        xaxis=dict(
            title="Costo Total (Miles de millones COP)",
            range=[0, df_plot["Costo_mil_millones"].max() * 1.15]
        ),
        yaxis=dict(autorange="reversed"),
        margin=dict(t=40, l=120, r=20, b=40)
    )

    return fig


# === 3. Heatmap (categoría de producto vs categoría de insumo) ===
def grafica_heatmap_insumos():
    df_heat = (
        df_maestro_2024.groupby(["Categoría Proc", "categoria_insumo"], as_index=False)["CostoTotalInsumo"]
        .sum()
    )

    # Calcular el total por categoría de producto
    total_por_categoria = df_heat.groupby("Categoría Proc")["CostoTotalInsumo"].transform("sum")

    # Calcular el porcentaje de participación
    df_heat["Porcentaje"] = (df_heat["CostoTotalInsumo"] / total_por_categoria) * 100

    # Crear el heatmap con el porcentaje
    fig = px.density_heatmap(
        df_heat,
        x="Categoría Proc",
        y="categoria_insumo",
        z="Porcentaje",
        color_continuous_scale="Viridis",
        title="Porcentaje de participación de insumos por categoría de producto"
    )

    fig.update_layout(
        xaxis_title="Categoría de Producto",
        yaxis_title="Categoría de Insumo",
        margin=dict(t=40, l=80, r=20, b=40)
    )

    return fig

grafica_heatmap_insumos().show()
grafica_treemap_insumos().show()
grafica_top_insumos().show()

### Rentabilidad

In [28]:
# 1️⃣ Boxplot / Violin plot por categoría de producto
def grafica_rentabilidad_boxplot():
    fig = px.box(
        df_maestro_2024,
        x="Categoría Proc",
        y="Margen Ganancia Real",
        color="Categoría Proc",
        # points="all",  # muestra también puntos individuales
        title="Distribución de la Rentabilidad por Categoría de Producto"
    )
    fig.update_layout(
        xaxis_title="Categoría de Producto",
        yaxis_title="Margen de Ganancia Real (%)",
        legend_title="Categoría"
    )
    return fig


# 2️⃣ Scatter Plot Costo vs Precio Unitario
def grafica_costo_vs_precio():
    fig = px.scatter(
        df_maestro_2024,
        x="Costo Total Unitario",
        y="Precio Unitario Sin IVA",
        color="Categoría Proc",
        hover_data=["ClienteNombre", "Producto", "Ganancia", "Margen Ganancia Real"],
        title="Relación entre Costo Unitario y Precio de Venta"
    )
    fig.update_layout(
        xaxis_title="Costo Total Unitario",
        yaxis_title="Precio Unitario Sin IVA",
        legend_title="Categoría"
    )
    return fig


# 3️⃣ Evolución mensual de rentabilidad promedio
def grafica_rentabilidad_mensual():
    df_mensual = (
        df_maestro_2024
        .groupby(df_maestro_2024["Creado El"].dt.to_period("M"))
        ["Margen Ganancia Real"].mean()
        .reset_index()
    )
    df_mensual["Creado El"] = df_mensual["Creado El"].dt.to_timestamp()

    fig = px.line(
        df_mensual,
        x="Creado El",
        y="Margen Ganancia Real",
        markers=True,
        title="Evolución Mensual del Margen de Ganancia Promedio"
    )
    
    # Asegurar etiquetas por mes en el eje X
    fig.update_layout(
        xaxis=dict(
            title="Mes",
            dtick="M1",  # Intervalo de 1 mes
            tickformat="%b %Y",  # Formato: abreviatura del mes y año
        ),
        yaxis_title="Margen de Ganancia Promedio (%)"
    )
    
    return fig

In [29]:
grafica_rentabilidad_boxplot().show()
grafica_costo_vs_precio().show()
grafica_rentabilidad_mensual().show()

In [30]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def grafica_dashboard_mensual():
    # Preparación de datos mensuales
    df_mensual = (
        df_maestro_2024
        .groupby(df_maestro_2024["Creado El"].dt.to_period("M"))
        .agg({
            "Total Pieza": "sum",
            "CostoTotalInsumo": "sum",
            "Margen Ganancia Real": "mean"
        })
        .reset_index()
    )
    df_mensual["Creado El"] = df_mensual["Creado El"].dt.to_timestamp()

    # Subplots (3 filas, 1 columna)
    fig = make_subplots(
        rows=3, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.08,
        subplot_titles=("Volumen producido", "Costo de Materias Primas", "Rentabilidad Promedio")
    )

    # 1️⃣ Volumen producido
    fig.add_trace(
        go.Scatter(
            x=df_mensual["Creado El"],
            y=df_mensual["Total Pieza"],
            mode="lines+markers",
            name="Volumen"
        ),
        row=1, col=1
    )

    # 2️⃣ Costo de materias primas
    fig.add_trace(
        go.Scatter(
            x=df_mensual["Creado El"],
            y=df_mensual["CostoTotalInsumo"],
            mode="lines+markers",
            name="Costo Materias Primas"
        ),
        row=2, col=1
    )

    # 3️⃣ Rentabilidad promedio
    fig.add_trace(
        go.Scatter(
            x=df_mensual["Creado El"],
            y=df_mensual["Margen Ganancia Real"],
            mode="lines+markers",
            name="Rentabilidad"
        ),
        row=3, col=1
    )

    # Layout general
    fig.update_layout(
        height=900,
        title="Dashboard Mensual Consolidado (Producción, Costos y Rentabilidad)",
        showlegend=False,
    )

    # Configurar el eje X para mostrar etiquetas mensuales
    fig.update_xaxes(
        title_text="Mes",
        dtick="M1",  # Intervalo de 1 mes
        tickformat="%b %Y",  # Formato: abreviatura del mes y año
        row=3, col=1
    )
    fig.update_yaxes(title_text="Volumen (piezas)", row=1, col=1)
    fig.update_yaxes(title_text="Costo Total (moneda)", row=2, col=1)
    fig.update_yaxes(title_text="Rentabilidad (%)", row=3, col=1)

    return fig

In [31]:
grafica_dashboard_mensual().show()


In [32]:
df_maestro_2024.to_parquet("data/df_maestro_2024.parquet", index=False)

## Exportación reporte HTML

In [ ]:
from plotly.io import write_html
import os

def export_dashboard():
    # Crear un directorio para guardar el archivo HTML si no existe
    output_dir = "dashboards"
    os.makedirs(output_dir, exist_ok=True)
    
    # Crear un archivo HTML con las gráficas
    with open(os.path.join(output_dir, "index.html"), "w", encoding="utf-8") as f:
        f.write("""<!DOCTYPE html>\n<html lang='en'>\n<head>\n<meta charset='UTF-8'>\n<meta name='viewport' content='width=device-width, initial-scale=1.0'>\n<title>Dashboard</title>\n</head>\n<body>\n<h1>Dashboard Consolidado</h1>\n""")
        
        # Volúmenes de producción
        f.write("<h2>Volumenes de Producción</h2>\n")
        grafica_volumenes_fig = grafica_volumenes()
        write_html(grafica_volumenes_fig, file=f, full_html=False, include_plotlyjs='cdn')
        grafica_voluenes_bajo_total_fig = grafica_voluenes_bajo_total()
        write_html(grafica_voluenes_bajo_total_fig, file=f, full_html=False, include_plotlyjs='cdn')

        # Materias primas e insumos
        f.write("<h2>Distribución de Costos por Insumos / Materias Primas / Servicios de Producción</h2>\n")
        fig_heatmap = grafica_heatmap_insumos()
        write_html(fig_heatmap, file=f, full_html=False, include_plotlyjs='cdn')
        fig_treemap = grafica_treemap_insumos()
        write_html(fig_treemap, file=f, full_html=False, include_plotlyjs='cdn')
        fig_top_insumos = grafica_top_insumos()
        write_html(fig_top_insumos, file=f, full_html=False, include_plotlyjs='cdn')
        
        # Rentabilidad
        f.write("<h2>Rentabilidad</h2>\n")
        fig_boxplot = grafica_rentabilidad_boxplot()
        write_html(fig_boxplot, file=f, full_html=False, include_plotlyjs='cdn')
        fig_scatter = grafica_costo_vs_precio()
        write_html(fig_scatter, file=f, full_html=False, include_plotlyjs='cdn')
        fig_rentabilidad_mensual = grafica_rentabilidad_mensual()
        write_html(fig_rentabilidad_mensual, file=f, full_html=False, include_plotlyjs='cdn')
        fig_dashboard = grafica_dashboard_mensual()
        write_html(fig_dashboard, file=f, full_html=False, include_plotlyjs='cdn')
        
        f.write("</body>\n</html>")
    
    print(f"Dashboard exportado exitosamente en: {os.path.join(output_dir, 'index.html')}")
    
# Llamar a la función para exportar el dashboard
export_dashboard()

Dashboard exportado exitosamente en: exported_dashboards\index.html
